The goal of this work is to process a text dataset using Neural Networks and Deep Learning
word embedding and data analytics methods and to extract knowledge from it. Prepare a report
for this work and deposit it on moodle.

In this work you will use 20 Newsgroup dataset, but you a free to use any text data (UCI datasets
repository, kaggle, data.gouv.fr, …) informing the Professor.

The work should contains at least the following 4 parts:
1. Analysis of the text dataset
2. Text processing and Transformation
3. Apply di erent Neural Networks (NN) embedding techniques
4. Clustering and/or classi cation on the embedded data
5. Results analysis and visualisation
6. Theoretical formalism

In [206]:
# In this work you will use 20 Newsgroup dataset
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
import os
import sys
import time
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import TfidfVectorizer

In [207]:
# Analyse the dataset : the context, size, difficulties, detect the objectives.

categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

# Load the 20 newsgroups dataset
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

# the context of the dataset
print(newsgroups_train.target_names)
print(newsgroups_train.data[0])


['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']
From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton
Organization: The City University
Lines: 14

Does anyone know of a good way (standard PC application/PD utility) to
convert tif/img/tga files into LaserJet III format.  We would also like to
do the same, converting to HPGL (HP plotter) files.

Please email any response.

Is this the correct group?

Thanks in advance.  Michael.
-- 
Michael Collier (Programmer)                 The Computer Unit,
Email: M.P.Collier@uk.ac.city                The City University,
Tel: 071 477-8000 x3769                      London,
Fax: 071 477-8565                            EC1V 0HB.



In [208]:
# analyse the size of the dataset
print(len(newsgroups_train.data))
print(len(newsgroups_test.data))

2257
1502


In [209]:
# Text Processing and Transformation
# For this part, you should use scikit-learn and you can follow the tutorial:
# https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html#tutorial-setup

# Assign a fixed integer id to each word occurring in any document of the training set (for instance by building a dictionary from words to integer indices)


def build_dictionary(data):
    dictionary = {}
    for doc in data:
        for word in doc.split():
            if word not in dictionary:
                dictionary[word] = len(dictionary)
    return dictionary


data = newsgroups_train.data
dictionary = build_dictionary(newsgroups_train.data)


# For each document #i, count the number of occurrences of each word w and store it in X[i, j] as the value of feature #j where j is the index of word w in the dictionary
def build_X(data, dictionary):
    X = np.zeros((len(data), len(dictionary)), dtype=np.int)
    for i, doc in enumerate(data):
        for word in doc.split():
            X[i, dictionary[word]] += 1
    return X

X = build_X(data, dictionary)

In [210]:
# Tokenizing text with scikit-learn


def tokenize(data, dictionary):
    vectorizer = CountVectorizer(vocabulary=dictionary)
    X = vectorizer.fit_transform(data)
    return X


X = tokenize(newsgroups_train.data, dictionary)
print(X)

  (0, 6)	1
  (0, 7)	4
  (0, 12)	1
  (0, 18)	1
  (0, 20)	1
  (0, 21)	1
  (0, 22)	1
  (0, 24)	1
  (0, 25)	1
  (0, 30)	1
  (0, 32)	2
  (0, 33)	1
  (0, 37)	1
  (0, 38)	1
  (0, 39)	1
  (0, 40)	1
  (0, 41)	5
  (0, 43)	2
  (0, 49)	2
  (0, 50)	1
  (0, 53)	1
  (0, 54)	1
  (0, 57)	1
  (0, 70)	2
  (0, 72)	1
  :	:
  (2256, 2743)	1
  (2256, 2814)	1
  (2256, 3446)	1
  (2256, 3515)	1
  (2256, 6668)	1
  (2256, 9242)	1
  (2256, 9881)	1
  (2256, 12086)	1
  (2256, 16272)	1
  (2256, 18042)	1
  (2256, 18745)	1
  (2256, 18828)	2
  (2256, 19531)	1
  (2256, 20240)	2
  (2256, 21061)	1
  (2256, 23099)	1
  (2256, 25390)	3
  (2256, 28193)	1
  (2256, 32370)	1
  (2256, 32371)	1
  (2256, 32372)	1
  (2256, 36465)	1
  (2256, 38349)	2
  (2256, 39964)	1
  (2256, 63038)	2


In [211]:
def tfidf(X):
    transformer = TfidfTransformer()
    X = transformer.fit_transform(X)
    return X


X = tfidf(X)

print(X)

  (0, 66443)	0.33382329100385516
  (0, 66168)	0.1600976026643201
  (0, 51318)	0.23512011093968901
  (0, 40868)	0.11691909504979228
  (0, 40140)	0.16183455171633535
  (0, 32452)	0.13840804302402393
  (0, 25953)	0.4258541770474272
  (0, 24016)	0.04882171577232993
  (0, 18419)	0.13268998881837948
  (0, 14034)	0.06650905416446005
  (0, 12086)	0.02284345936023963
  (0, 11588)	0.09029694329795517
  (0, 11233)	0.13536985796475323
  (0, 8479)	0.11396004951458882
  (0, 7363)	0.08895769592383071
  (0, 6675)	0.10418638634047582
  (0, 5458)	0.0930013435412932
  (0, 2793)	0.07337861698182968
  (0, 2457)	0.0930013435412932
  (0, 2428)	0.12614447650727276
  (0, 2399)	0.046446684144222664
  (0, 2376)	0.022752590230327396
  (0, 1592)	0.09344748705583601
  (0, 1334)	0.024059020285550552
  (0, 640)	0.06402924290015559
  :	:
  (2256, 1099)	0.07496228611107417
  (2256, 1053)	0.10100422034712904
  (2256, 864)	0.1290659155273912
  (2256, 675)	0.061666226927635195
  (2256, 527)	0.05039405500374404
  (2256, 49

In [212]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


def build_doc2vec(data):
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(data)]
    model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)
    return model


model = build_doc2vec(newsgroups_train.data)


def build_doc2vec_X(data, model):
    X = np.zeros((len(data), 5))
    for i, doc in enumerate(data):
        X[i] = model.infer_vector(doc.split())
    return X


X_Doc2Vec = build_doc2vec_X(newsgroups_train.data, model)
print(X_Doc2Vec)

[[ 0.04773632 -0.0098836  -0.15603225 -0.04076122  0.0950311 ]
 [-0.27380687  0.02041226 -0.35173982 -0.06238876  0.05226387]
 [-0.63629282  0.0140928   0.04450047 -0.06471214  0.01463419]
 ...
 [-0.78709114 -0.18422848 -0.7816602   0.09781992  0.68411493]
 [-1.06335747 -0.12192961 -0.06953645  0.15266028  0.13134892]
 [-0.28968838 -0.06417686 -0.18007387  0.09793076  0.19547325]]


In [213]:
# BERT model
def build_bert_X(data):
    from transformers import BertTokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    X = np.zeros((len(data), 768))
    for i, doc in enumerate(data):
        X[i] = tokenizer.encode(doc, add_special_tokens=True, max_length=768, pad_to_max_length=True)
    return X


X_bert = build_bert_X(newsgroups_train.data)
print(X_bert)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[[ 101. 2013. 1024. ...    0.    0.    0.]
 [ 101. 2013. 1024. ...    0.    0.    0.]
 [ 101. 2013. 1024. ...    0.    0.    0.]
 ...
 [ 101. 2013. 1024. ... 1012. 4895.  102.]
 [ 101. 2013. 1024. ... 5653. 2075.  102.]
 [ 101. 2013. 1024. ...    0.    0.    0.]]
